# Header

preprocess to generate `/data/GC_output/analysis/percent2` dataset

forked from `preprocess.ipynb`

In [2]:
import glob
import re
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
import his_preprocess

# Only avg file

In [4]:
# GC 데이터 전처리


# {gaussian scale}_{# values}_{ens i}.nc

scale_list = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5]

file_dic ={}
for target_var in ["2m_temperature"]:

    if target_var == '2m_temperature':
        # file_dic[0] = sorted(glob.glob('/data/GC_output/percent2/GC_11111111111_250_*.nc'))
        for i, scale in enumerate(scale_list):
            file_dic[i] = sorted(glob.glob(f'/geodata2/S2S/DL/GC_input/shuffle/ERA5_{scale}_*_0.nc'))
            #print(file_dic[i])

    # Assign base colors for each partition
    colors_list = [
        '#008000', # 초록
        '#FF00FF', # 마젠타
        '#0000FF', # 파랑
        '#800080', # 보라
        '#FF69B4', # 핫핑크
        '#FFD700', # 골드
        '#FF0000', # 빨강
        '#00FF00', # 라임
        '#00FFFF', # 시안
        '#FFA500', # 주황
        '#800000', # 마룬
        '#4B0082', # 인디고
        '#8B4513', # 새들브라운
        '#FF4500', # 오렌지레드
        '#00FF7F', # 스프링그린
        '#1E90FF', # 도저블루
        '#FF1493', # 딥핑크
        '#7B68EE', # 미디엄슬레이트블루
        '#20B2AA', # 라이트시그린
        '#DAA520' # 골든로드
    ]

    def extract_perturbation_info(filename):
        match = filename.split("/")[-1].split("_")
        if match:
            scale = match[1]  # perturbation 코드
            percent = match[2]  # 값 (예: 0.001)
            ens = match[3]  # 지역 코드 (예: 9p)
            #print(f"{scale}_{percent}_{ens}")
            return f"{scale}_{percent}_{ens}"
        else:
            return None

    # Collect perturbation files with labels and colors
    perturb_files = []
    
    for i, partition_files in file_dic.items():
        base_color = colors_list[i % len(colors_list)]
        num_files = len(partition_files)
        # Generate different shades of the base color
        colors = sns.light_palette(base_color, n_colors=num_files + 2)[1:-1]
        for i, file in enumerate(partition_files):
            perturb_info = extract_perturbation_info(file)
            if perturb_info:
                label = f"{perturb_info}"
                color = colors[i % len(colors_list)]
                perturb_files.append((label, color, file))

    # perturb_datasets_raw = []
    perturb_datasets_mean = []

    def process_file(file_info):
        label, color, file = file_info
        print(f"processing {file}")
        dataset_raw = his_preprocess.preprocess_GC(xr.open_dataset(file, engine="netcdf4"), target_var)
        # return {"mean": (label, color, his_preprocess.weighted_mean(dataset_raw)), "raw": (label, color, dataset_raw)}
        return {"mean": (label, color, his_preprocess.weighted_mean(dataset_raw))}

    with Pool() as executor:
        results = executor.map(process_file, perturb_files)

    perturb_datasets_mean = [result["mean"] for result in results]
    # perturb_datasets_raw = [result["raw"] for result in results]

    if target_var == '2m_temperature':
        with open('/data/GC_output/analysis/shuffle/ERA5_t2m_GlobAvg.pkl', 'wb') as f:
            pickle.dump(perturb_datasets_mean, f)
        # with open('/data/GC_output/analysis/shuffle/sparse_GC_t2m_Globraw.pkl', 'wb') as f:
        #     pickle.dump(perturb_datasets_raw, f)

processing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.5_9_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.5_1_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.3_9_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.01_9_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.1_5_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.3_1_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.4_9_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.01_7_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.1_1_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.5_3_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.1_9_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.4_5_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.01_5_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.5_5_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.2_3_0.ncprocessing /geodata2/S2S/DL/GC_input/shuffle/ERA5_0.3_7_0.ncprocessing /geodata2/S2S/DL/GC_input/

ValueError: conflicting sizes for dimension 'time': length 10 on 'time' and length 1 on {'lat': 'lat', 'lon': 'lon', 'level': 'level', 'time': '10m_u_component_of_wind'}

# Only Raw File

In [3]:
# GC 데이터 전처리

# {gaussian scale}_{# values}_{ens i}.nc

scale_list = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5]

file_dic ={}
for target_var in ["2m_temperature"]:

    if target_var == '2m_temperature':
        # file_dic[0] = sorted(glob.glob('/data/GC_output/percent2/GC_11111111111_250_*.nc'))
        for i, scale in enumerate(scale_list):
            file_dic[i] = sorted(glob.glob(f'/data/GC_output/shuffle/GC_{scale}_*_*.nc'))
            # print(file_dic[i])

    # Assign base colors for each partition
    colors_list = [
        '#008000', # 초록
        '#FF00FF', # 마젠타
        '#0000FF', # 파랑
        '#800080', # 보라
        '#FF69B4', # 핫핑크
        '#FFD700', # 골드
        '#FF0000', # 빨강
        '#00FF00', # 라임
        '#00FFFF', # 시안
        '#FFA500', # 주황
        '#800000', # 마룬
        '#4B0082', # 인디고
        '#8B4513', # 새들브라운
        '#FF4500', # 오렌지레드
        '#00FF7F', # 스프링그린
        '#1E90FF', # 도저블루
        '#FF1493', # 딥핑크
        '#7B68EE', # 미디엄슬레이트블루
        '#20B2AA', # 라이트시그린
        '#DAA520' # 골든로드
    ]

    def extract_perturbation_info(filename):
        match = filename.split("/")[-1].split("_")
        if match:
            scale = match[1]  # perturbation 코드
            percent = match[2]  # 값 (예: 0.001)
            ens = match[3]  # 지역 코드 (예: 9p)
            # print(f"{scale}_{percent}_{ens}")
            return f"{scale}_{percent}_{ens}"
        else:
            return None

    # Collect perturbation files with labels and colors
    perturb_files = []
    
    for i, partition_files in file_dic.items():
        base_color = colors_list[i % len(colors_list)]
        num_files = len(partition_files)
        # Generate different shades of the base color
        colors = sns.light_palette(base_color, n_colors=num_files + 2)[1:-1]
        for i, file in enumerate(partition_files):
            perturb_info = extract_perturbation_info(file)
            if perturb_info:
                label = f"{perturb_info}"
                color = colors[i % len(colors_list)]
                perturb_files.append((label, color, file))

    perturb_datasets_raw = []
    perturb_datasets_mean = []

    def process_file(file_info):
        label, color, file = file_info
        dataset_raw = his_preprocess.preprocess_GC(xr.open_dataset(file, engine="netcdf4"), target_var)
        # return {"mean": (label, color, his_preprocess.weighted_mean(dataset_raw)), "raw": (label, color, dataset_raw)}
        return {"raw": (label, color, dataset_raw)}

    with Pool(processes=13) as executor:
        results = executor.map(process_file, perturb_files)

    # perturb_datasets_mean = [result["mean"] for result in results]
    perturb_datasets_raw = [result["raw"] for result in results]

    if target_var == '2m_temperature':
        # with open('/data/GC_output/analysis/shuffle/full_GC_t2m_GlobAvg.pkl', 'wb') as f:
        #     pickle.dump(perturb_datasets_mean, f)
        with open('/data/GC_output/analysis/shuffle/full_GC_t2m_Globraw.pkl', 'wb') as f:
            pickle.dump(perturb_datasets_raw, f)